In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tkinter import Tk, Label, Entry, Button, Listbox, END
from nltk.stem.porter import PorterStemmer

# Preprocess your dataset
ps = PorterStemmer()

def stem(text):
    return " ".join(ps.stem(word) for word in text.split())

# Load and prepare data
movies = pd.DataFrame({
    'id_x': [19995, 285, 206647, 49026, 49529],
    'title': ['Avatar', 'Pirates of the Caribbean: At World\'s End', 
              'Spectre', 'The Dark Knight Rises', 'John Carter'],
    'tags': [
        'in the 22nd century paraplegic marine action adventure fantasy',
        'captain barbossa believed dead adventure fantasy action',
        'cryptic message bond past action adventure crime',
        'district attorney harvey death action crime drama thriller',
        'john carter war weary action adventure science fiction'
    ]
})
movies['tags'] = movies['tags'].apply(lambda x: x.lower())
movies['tags'] = movies['tags'].apply(stem)

# Generate vectors and similarity matrix
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(movies['tags']).toarray()
similarity = cosine_similarity(vectors)

# Recommendation function
def recommend(movie_name):
    try:
        movie_index = movies[movies['title'] == movie_name].index[0]
        distances = similarity[movie_index]
        movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
        return [movies.iloc[i[0]].title for i in movie_list]
    except IndexError:
        return ["Movie not found!"]

# GUI Application
def show_recommendations():
    movie_name = movie_entry.get()
    recommendations = recommend(movie_name)
    recommendations_list.delete(0, END)
    for rec in recommendations:
        recommendations_list.insert(END, rec)

# Tkinter GUI
root = Tk()
root.title("Movie Recommendation System")

Label(root, text="Enter Movie Name:").grid(row=0, column=0, padx=10, pady=10)
movie_entry = Entry(root, width=40)
movie_entry.grid(row=0, column=1, padx=10, pady=10)

Button(root, text="Recommend", command=show_recommendations).grid(row=1, column=0, columnspan=2, pady=10)

Label(root, text="Recommendations:").grid(row=2, column=0, padx=10, pady=10)
recommendations_list = Listbox(root, width=50, height=10)
recommendations_list.grid(row=2, column=1, padx=10, pady=10)

root.mainloop()
